# Descarga de Dados GFS



## Instalación de librerias necesarias

- Librería siphon


```bash
conda install -c conda-forge geemap eemont siphon xarray[complete] netCDF4
```


In [76]:
from siphon.catalog import TDSCatalog

# si el catalogo que se desea es de 0.5 grados
# best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml')

# si el catalogo que se desea es de 0.25 grados
best_gfs = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/catalog.xml')

# lista de colecciones
best_gfs.datasets

['Full Collection (Reference / Forecast Time) Dataset', 'Best GFS Quarter Degree Forecast Time Series', 'Latest Collection for GFS Quarter Degree Forecast']

In [77]:
best_ds = list(best_gfs.datasets.values())[1]

# diccionario de urls disponibles con la información
best_ds.access_urls

{'OPENDAP': 'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p25deg/Best',
 'CdmRemote': 'https://thredds.ucar.edu/thredds/cdmremote/grib/NCEP/GFS/Global_0p25deg/Best',
 'JupyterNotebook': 'https://thredds.ucar.edu/thredds/notebook/grib/NCEP/GFS/Global_0p25deg/Best',
 'NetcdfSubset': 'https://thredds.ucar.edu/thredds/ncss/grid/grib/NCEP/GFS/Global_0p25deg/Best',
 'WMS': 'https://thredds.ucar.edu/thredds/wms/grib/NCEP/GFS/Global_0p25deg/Best',
 'WCS': 'https://thredds.ucar.edu/thredds/wcs/grib/NCEP/GFS/Global_0p25deg/Best',
 'ISO': 'https://thredds.ucar.edu/thredds/iso/grib/NCEP/GFS/Global_0p25deg/Best',
 'NCML': 'https://thredds.ucar.edu/thredds/ncml/grib/NCEP/GFS/Global_0p25deg/Best',
 'UDDC': 'https://thredds.ucar.edu/thredds/uddc/grib/NCEP/GFS/Global_0p25deg/Best'}

In [78]:
# utilizando el servicio de netcdf-subset

from siphon.ncss import NCSS
ncss = NCSS(best_ds.access_urls['NetcdfSubset'])

In [79]:
# generando un sistema de consulta en el netcdf-subset
query = ncss.query()

In [1]:
from siphon.catalog import TDSCatalog
import xarray as xr
import datetime

In [2]:
# Latest GFS TDSCatalog
url = 'https://thredds-jumbo.unidata.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p25deg/latest.xml'
catalog = TDSCatalog(url)
print(catalog.datasets)

['GFS_Global_0p25deg_20221022_1200.grib2']


In [3]:
# Time of interest
obs = datetime.date.today() + datetime.timedelta(days = 1)
obs_time = datetime.datetime(obs.year, obs.month, obs.day,
                             hour = 6, minute = 0)
obs_time

datetime.datetime(2022, 10, 23, 6, 0)

In [4]:
def get_netcdf_subset(catalog, variable, obs_time):
    # Boundaries
    south = -15.0
    north = 15.0
    west = 90
    east = 150
    
    # NetCDF Subset Service Query
    ncss = catalog.datasets[0].subset()
    query = ncss.query()
    
    query.lonlat_box(west = west, east = east, south = south, north = north)
    query.variables(variable)
    query.time(obs_time)

    return xr.open_dataset(xr.backends.NetCDF4DataStore(ncss.get_data(query)))


In [67]:
subss = catalog.datasets[0].subset()

query = subss.query()

query.lonlat_box(west=-83, east=-67, north=0, south=-20)

query.accept('netcdf4')

query.variables('Air_temperature')

var=Air_temperature&west=-83&east=-67&south=-20&north=0&accept=netcdf4

In [68]:
data = subss.get_data(query)
list(data.variables)

HTTPError: Error accessing https://thredds-jumbo.unidata.ucar.edu/thredds/ncss/grid/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_20221022_1200.grib2?var=Air_temperature&west=-83&east=-67&south=-20&north=0&accept=netcdf4
Server Error (400: Variable: Air_temperature is not contained in the requested dataset)

In [27]:
rain = 'Total_precipitation_surface_Mixed_intervals_Accumulation'

ds_rain = get_netcdf_subset(catalog, rain, obs_time)
print(type(ds_rain))

# verificar los nombres de las variables y dimensiones
print(ds_rain.keys())

# buscar el nombre con el que se guarda la variable tiempo
ds_rain = get_netcdf_subset(catalog, rain, obs_time).sel(time2 = obs_time)

# ds_rain

<class 'xarray.core.dataset.Dataset'>
KeysView(Frozen({'time2_bounds': <xarray.Variable (time2: 173, bounds_dim: 2)>
array([['2022-10-23T00:00:00.000000000', '2022-10-23T06:00:00.000000000'],
       ['2022-10-22T12:00:00.000000000', '2022-10-23T09:00:00.000000000'],
       ['2022-10-23T06:00:00.000000000', '2022-10-23T09:00:00.000000000'],
       ...,
       ['2022-11-06T18:00:00.000000000', '2022-11-07T00:00:00.000000000'],
       ['2022-10-22T12:00:00.000000000', '2022-11-07T12:00:00.000000000'],
       ['2022-11-07T06:00:00.000000000', '2022-11-07T12:00:00.000000000']],
      dtype='datetime64[ns]'), 'time2': <xarray.IndexVariable 'time2' (time2: 173)>
array(['2022-10-23T03:00:00.000000000', '2022-10-22T22:30:00.000000000',
       '2022-10-23T07:30:00.000000000', '2022-10-23T00:00:00.000000000',
       '2022-10-23T09:00:00.000000000', '2022-10-23T01:30:00.000000000',
       '2022-10-23T13:30:00.000000000', '2022-10-23T03:00:00.000000000',
       '2022-10-23T15:00:00.000000000', '202

In [21]:
temp = 'Temperature_surface'
ds_temp = get_netcdf_subset(catalog, temp, obs_time).sel(time3 = obs_time)

In [23]:
wind = 'Wind_speed_gust_surface'
ds_wind = get_netcdf_subset(catalog, wind, obs_time).sel(time3 = obs_time)

In [26]:
rhum = 'Relative_humidity_isaboric'
ds_rhum = get_netcdf_subset(catalog, rhum, obs_time) \
    .sel(height_above_ground = 2.0) \
    .sel(time3 = obs_time)

HTTPError: Error accessing https://thredds-jumbo.unidata.ucar.edu/thredds/ncss/grid/grib/NCEP/GFS/Global_0p25deg/GFS_Global_0p25deg_20221022_1200.grib2?var=Relative_humidity_isaboric&time=2022-10-23T06%3A00%3A00&west=90&east=150&south=-15.0&north=15.0
Server Error (400: Variable: Relative_humidity_isaboric is not contained in the requested dataset)

In [ ]:
# print(ds_rain.keys)
ds_rain.Total_precipitation_surface_Mixed_intervals_Accumulation.plot(x='longitude', y='latitude')
